In [ ]:
import numpy as np
import pandas as pd

In [ ]:
hate_speech = pd.read_csv('/notebooks/labeled_data.csv')
hate_speech

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [ ]:
orig_list = list(hate_speech['tweet'])
list_eng = orig_list[:1000]
len(list_eng)

1000

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.5 MB/s eta 0:00:00


In [ ]:
import transformers
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
from collections import OrderedDict

vowels = OrderedDict([
('ँ','n'),
('ं','n'),
('ः','a'),
('अ','a'),
('आ','aa'),
('इ','i'),
('ई','ee'),
('उ','u'),
('ऊ','oo'),
('ऋ','ri'),
('ए','e'),
('ऐ','ae'),
('ओ','o'),
('औ','au'),
('ा','a'),
('ि','i'),
('ी','i'),
('ु','u'),
('ू','oo'),
('ृ','ri'),
('े','e'),
('ै','ai'),
('ो','o'),
('ौ','au')
])

consonants = OrderedDict([
('क','k'),
('ख','kh'),
('ग','g'),
('घ','gh'),
('ङ','ng'),

('च','ch'),
('छ','chh'),
('ज','j'),
('ज़','z'),
('ज़','z'), #these two are very different, see them in unicode by 'ज़'.encode('utf-8'). You'll see.
('झ','jh'),
('ञ','nj'),

('ट','t'),
('ठ','th'),
('ड','d'),
('ड़','r'),
('ड़','r'), #these two are very different, see them in unicode by 'ड़'.encode('utf-8'). You'll see.
('ढ','dh'),
('ण','n'),

('त','t'),
('थ','th'),
('द','d'),
('ध','dh'),
('न','n'),

('प','p'),
('फ','ph'),
('फ़','f'),
('फ़','f'), #these two फ़ are very different, see them in unicode by 'फ़'.encode('utf-8'). You'll see.
('ब','b'),
('भ','bh'),
('म','m'),

('य','y'),
('र','r'),
('ल','l'),
('व','v'),
('श','sh'),

('ष','sh'),
('स','s'),
('ह','h'),
('क्ष','ksh'),
('त्र','tr'),
('ज्ञ','gy')
	])

with open('/notebooks/svar.csv', 'w') as f:
	csvwriter = csv.writer(f)
	for k,v in zip(vowels.keys(), vowels.values()):
		csvwriter.writerow([k,v])

with open('/notebooks/vyanjan.csv', 'w') as f:
	csvwriter = csv.writer(f)
	for k,v in zip(consonants.keys(), consonants.values()):
		csvwriter.writerow([k,v])

In [ ]:
# -*- coding: utf-8 -*-
import csv
import sys

reader = csv.reader(open('/notebooks/svar.csv', 'r'))
vowels = {}
for row in reader:
	k, v = row
	vowels[k] = v

reader = csv.reader(open('/notebooks/vyanjan.csv', 'r'))
consonants = {}
for row in reader:
	k, v = row
	consonants[k] = v

In [ ]:
def convert_word_to_roman(content):
  str1 = ""
  for x in content:
    for y in x.split():
      for i in range(len(y)):
        if (i+1<len(y) and y[i+1].strip()==' ़'.strip()):
          c = y[i]+y[i+1]
          p=2
        else:
          c = y[i]
          p=1
        if (c in vowels.keys()):
          str1 = str1 + vowels[c]
        elif (c in consonants.keys()):
          if(i+p<len(y) and y[i+p] in consonants.keys()):
            if ((c=='झ' and i!=0) or (i!=0 and i+p+1<len(y) and y[i+p+1] in vowels.keys())): # add 'a' after 'jh', only if झ appears in the starting of the word
              str1 = str1 + consonants[c]
            else:
              str1 = str1 + consonants[c]+'a'
          else:
            str1 = str1 + consonants[c]
        elif y[i] in ['\n','\t',' ','!',',','।','-',':','\\','_','?'] or c.isalnum():
          str1 = str1 + c.replace('।','.')

  return str1

In [ ]:
def generate_hindi_from_english(sent):
  result = sent.split(" ")
  sent_tokenized = sent.split(" ")

  tokenizer.src_lang = "en_XX"
  encoded_hi = tokenizer(sent, return_tensors="pt")
  generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
  )
  res = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
  hindi_sub_list = res[0].split(" ")
  hindi = ""
  for i in hindi_sub_list:
    hindi = hindi + convert_word_to_roman(i) + " "
  print(hindi)
  return hindi

In [ ]:
list_hindi = []
count = 1
for i in list_eng:
  hindi = generate_hindi_from_english(i)
  print(count)
  count = count + 1
  list_hindi.append(hindi)

/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


!!! RT mayasolovely: ek mhila ke roop men aap apne ghr ki sphaee ke bare men shikayt nhin krna chahie amp ek aadmi ke roop men aap hmesha rddi le jana chahie 
1


/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


RT mleew17: Boy dats coldtyga dwn bad for cuffin dat hoe in the 1st place!! 
2
!!!!!!! RT UrKindOfBrand Dawg!!!! RT 80sbaby4life: kya tum kbhi ek billoo ko chooste ho aur vh rone lgta hai? 
3
!!!!!!!!! RT C_G_Anderson: viva_based vo ek trenni ki trh dikhti hai 
4
ShenikaRoberts: jo kuchh tum mere bare men sun rhe ho, vh sch ho skta hai ya vh us bdchln se bhi jyada jhootha ho skta hai, jisne ise tumse kha tha 
5
!!!!!!!!!!!!!!!!!!T_Madison_x: shooti bs mujhe phunkti hai tum itni eemandar aur kisi ke lie niche dava lekin abhi bhi hoos ke sath fuck! 
6
!!!!!!__BrighterDays: main sirph baithna nhin kr skta aur ek aur bdchln pr HATE main ja rhi hai bhut bdchln ho gya! 
7
selfiequeenbri: kyonki mujhe tum bdi boodhi boodhi ldkiyon ke lie aane se thk gya hai! 
8
amp tum tum bichhda vaps nhin mil skta hai amp ki hai ki  
9
 rhythmixx_ :hobbies include: fighting Mariam bitch 
10
kik ek bdchln hai vh hr ek ko vkr krta hai lol main is trh ki vartalap men chla gya Smh 
11
murd gangh apni gangh bhoo

In [ ]:
custom_df = pd.DataFrame()
custom_df['English'] = list_eng
custom_df['Hindi'] = list_hindi
custom_df.to_csv("/notebooks/hate_speech_1.csv")

In [ ]:
twitter = pd.read_csv('/notebooks/Twitter_Data.csv')
twitter

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [ ]:
twitter['clean_text']

0         when modi promised “minimum government maximum...
1         talk all the nonsense and continue all the dra...
2         what did just say vote for modi  welcome bjp t...
3         asking his supporters prefix chowkidar their n...
4         answer who among these the most powerful world...
                                ...                        
162975    why these 456 crores paid neerav modi not reco...
162976    dear rss terrorist payal gawar what about modi...
162977    did you cover her interaction forum where she ...
162978    there big project came into india modi dream p...
162979    have you ever listen about like gurukul where ...
Name: clean_text, Length: 162980, dtype: object

In [ ]:
list_eng = []
for i in range(10000, 10300):
    if twitter['category'][i] == -1.0:
        list_eng.append(twitter['clean_text'][i])

In [ ]:
list_eng

['only bjp under modi leadership india will prosper and become strong nation\ncorrupt kangress who looted india and indians should thrown into tihar jail vande mataram ',
 'when millions poor benefit from schemes you cannot say did not get and still keep saying that modi doing nothing for poor for water delhi has enough for its needs and haryana has been pretty cooperative',
 'these people should learn from modi and walk dead bodies make money',
 'economist favour also know that unfeasible india and mere poll gimmick thug rahul gandhi poll gimmick much like modis 2014 promise bring back black money from abroad',
 'fiscal viablity challenged corruptionmodi like maha haraaaaami corrupt looter can give lac crore package still nothing grt happened congress has delivered big promises ',
 'mamtime forget caste belief with due respectdo you believe therewe have leaders modikejriwalkaul brahmin rahulnadar kanimozhi with the whole country built based caste based reservationour dharma based puru

In [ ]:
e_l = []
for i in range(len(list_eng)):
  l_new = list_eng[i].split(".")
  for j in l_new:
    e_l.append(j)

In [ ]:
len(e_l)

72

In [ ]:
list_hindi = []
count = 1
for i in e_l:
  hindi = generate_hindi_from_english(i)
  print(count)
  count = count + 1
  list_hindi.append(hindi)

/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


sirph bhart ke netritv men bi bi pi ka vikas hoga aur shktishali rashtr bn jaega jo bhart ko loota tha aur bhartiyon ko thrir jel men phenkna chahie. 
1


/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


jb lakhon gribon ko yojnaon se labh milta hai to aap yh nhin kh skte ki unhen nhin mila aur phir bhi yh khte rhte hain ki pani ke lie gribon ke lie kuchh nhin kr rhi mdi apni jroorton ke lie pryapt hai aur hriyana kaphi shyogatmk rha hai 
2
in logon ko modi se sikhna chahie aur mrit shriron se paise kmane ke lie chlna chahie. 
3
arthshastri ka pksh yh bhi janta hai ki aprihary bhart aur kevl jnmt abhiyan khlnayk rahul gandhi jnmt abhiyan khlnayk modis 2014 ki bhanti  black money ko videshon se vaps lane ka vayda hai 
4
rajkoshiy sphlta ne bhrshtachar ko chunauti di mhamhim hrimoorti bhrshtachari luter ko lakh krod paikej de skta hai abhi bhi kuchh bhi nhin huaa kangres ne bdi vchn de diye hain. 
5
jb hm sbhi jati-vishvason ko uchit smman se bhulate hain, jb hm sbhi jati-vishvason ko uchit smman ke sath bhulate hain, jb hm sbhi jati-vishvason ko uchit smman ke sath bhulate hain, jb hm sbhi jati-vishvason ko uchit smman ke sath bhulate hain, jb hm sbhi jati-vishvason ko uchit smman ke sa

In [ ]:
custom_df = pd.DataFrame()
custom_df['English'] = e_l
custom_df['Hindi'] = list_hindi
custom_df.to_csv("/notebooks/twitter_5.csv")

In [ ]:
orig_list = list(jigsaw_val['text'])
list_eng = orig_list[200:250]
len(list_eng)
e_l = []
for i in range(len(list_eng)):
  l_new = list_eng[i].split(".")
  for j in l_new:
    e_l.append(j)
list_hindi = []
count = 1
for i in e_l:
  hindi = generate_hindi_from_english(i)
  print(count)
  count = count + 1
  list_hindi.append(hindi)
custom_df = pd.DataFrame()
custom_df['English'] = e_l
custom_df['Hindi'] = list_hindi
custom_df.to_csv("/content/drive/MyDrive/custom_roman_hindi_dataset_10.csv")

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


main un logon ke chehre pr ml chhidkana chahta hoon jo yh chahte hain. 
1
are! main behtr hai ki phadon ke lie bhagen! vaise hi, main koee vyvdhan nhin kiya 
2
di 
3
je 
4
braisi is trh anupat se bahr nikl gya 
5
dhvni 
6
beshk mainne tumhari tsvir dekhi hai, tum ek uchhlne vale ki trh lgte ho! apni muskan men achchhi ganthon! tumhare pas bde kan hain aur tumhari bal lgte hain 
7
ek tain le lo aur shayd aap apne pija chehre ke kuchh pimples kho denge! kya tum kbhi ek ldki se bat krte hain bina unhen apne kredit kard nnbr 1st diya hai!! bat 
8
mainne saindbks ka pryog kiya 
9
moorkh 
10
dhvni 
11
main poori trh shmt hoon 
12
is shbd ke bare men svabhavik roop se nindniy kuchh nhin hai 
13
redskin aur paile phes donon nslon ke bich sneh ki shrton ke roop men ek nishchit matra men upyog ka bhi aannd uthate hain 
14
mujhe lgta hai ki kuchh bhartiyon ko is shbd ko psnd nhin hai jaise kuchh, shayd isse bhi adhik smjhne ke lie, bhartiy shbd ko ghrina krte hain, jaise kuchh logon ko sphed, kal

In [ ]:
custom_df

,English,Hindi
0,I have the strong desire to smear feces all ov...,main un logon ke chehre pr ml chhidkana chahta...
1,Oh shit! I better flee for the hills! By the w...,are! main behtr hai ki phadon ke lie bhagen! v...
2,D,di
3,J,je
4,Bracey blew this way out of proportion,braisi is trh anupat se bahr nikl gya
...,...,...
391,"why should that not make us brothers? ( )""",hmen bhaee kyon nhin bnana chahie?
392,"""\n""""Actually, Jarlaxle is right; that shit wa...","vastv men, Jarlaxle shi hai, ki kooda mere dva..."
393,"Alex, you're a cunt","eleks, tum ek kooda ho"
394,'Nuff said,nph ne kha


In [ ]:
custom_df = pd.DataFrame()
custom_df['English'] = e_l
custom_df['Hindi'] = list_hindi
custom_df.to_csv("/content/drive/MyDrive/custom_roman_hindi_dataset_10.csv")